# Treinamento de uma CNN no CIFAR 10

**Nome: **


O objetivo deste exercício é implementar e treinar uma rede convolucional que obtenha a maior acurácia no dataset de teste do CIFAR 10.

Podem definir sua própria rede ou utilizar redes pré-treinadas, como a Resnet.

## Inicializando o Neptune

In [ ]:
! pip install neptune-client==0.9.1

In [ ]:
import neptune.new as neptune

# Insira seu api_token para logar os resultados do treino na sua conta do Neptune.
# Como obter seu API token do Neptune:
# https://docs.neptune.ai/administration/security-and-privacy/how-to-find-and-set-neptune-api-token

# run = neptune.init(project='rodrigonogueira/aula6', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhYzhlOTIxYy1lMGNhLTRiY2QtYTdjYi1jNWMyN2YxNzVhMTQifQ==')

## Importação das bibliotecas

In [ ]:
%matplotlib inline
import numpy as np

import torch
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import MNIST
from torchvision import transforms

import torchvision.models as models
torch.manual_seed(123)

## Download do Dataset

In [ ]:
!mkdir ./data

# referência https://pytorch.org/hub/pytorch_vision_mobilenet_v2/
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                             download=True, transform=preprocess)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=preprocess)


mkdir: cannot create directory ‘./data’: File exists



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Dataset e dataloader

In [ ]:
train_size = 40000
val_size = 10000
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [ ]:
batch_size = 50

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print('Número de minibatches de trenamento:', len(train_dataloader))
print('Número de minibatches de validação:', len(val_dataloader))
print('Número de minibatches de teste:', len(test_dataloader))

x_train, y_train = next(iter(train_dataloader))
print("\nDimensões dos dados de um minibatch:", x_train.size())
print("Valores mínimo e máximo dos pixels: ", torch.min(x_train), torch.max(x_train))
print("Tipo dos dados das imagens:         ", type(x_train))
print("Tipo das classes das imagens:       ", type(y_train))

Número de minibatches de trenamento: 800
Número de minibatches de validação: 200
Número de minibatches de teste: 200

Dimensões dos dados de um minibatch: torch.Size([50, 3, 224, 224])
Valores mínimo e máximo dos pixels:  tensor(-2.1179) tensor(2.6400)
Tipo dos dados das imagens:          <class 'torch.Tensor'>
Tipo das classes das imagens:        <class 'torch.Tensor'>


In [ ]:
# Use True, para usar o pytorch lightning original
pl_original = False

## Usando o Pytorch Lightining "SuperLight" (criado apenas para o curso).

Criamos um Pytorch Lightning "básico" que esperamos ser mais didático que o original pois o código é facil de entender caso ocorra algum erro.

As classes `LightningModule` e `Trainer` não precisam ser implementadas. Entretanto, para cada nova tarefa, uma classe que herda do `LightningModule` precisa ser definida e os seguintes métodos devem ser implementados:

 - \_\_init\_\_
 - forward
 - train_step
 - train_epoch_end
 - validation_step
 - validation_epoch_end
 - configure_optimizers

Os métodos `test_step` e `test_epoch_end` devem ser implementados apenas se trainer.test() for chamado.


In [ ]:
import abc


class LightningModule:
    @abc.abstractmethod
    def __init(self):
        return

    @abc.abstractmethod
    def forward(self):
        return

    @abc.abstractmethod    
    def training_step(self):
        return 

    @abc.abstractmethod    
    def training_epoch_end(self):
        return 

    @abc.abstractmethod    
    def validation_step(self):
        return 

    @abc.abstractmethod    
    def validation_epoch_end(self):
        return 

    @abc.abstractmethod    
    def test_step(self):
        return 

    @abc.abstractmethod    
    def test_epoch_end(self):
        return

    @abc.abstractmethod
    def configure_optimizers(self):
        return

In [ ]:
class Trainer():
    def __init__(self, max_epochs: int, gpus: int = 1):
        self.max_epochs = max_epochs
        dev = "cpu" 
        if gpus > 0:
            if torch.cuda.is_available(): 
                dev = "cuda:0"

        print(f'Using {dev}')
        self.device = torch.device(dev)

    def fit(self, model, train_dataloader, val_dataloader=None):
        assert isinstance(model, LightningModule)
        best_valid_loss = 10e9
        optimizers, _ = model.configure_optimizers()
        optimizer = optimizers[0]
        model.model.to(self.device)
        
        for i in range(self.max_epochs):
            outputs = []
            model.model.train()
            for batch_idx, (x_train, y_train) in enumerate(train_dataloader):
                x_train = x_train.to(self.device)
                y_train = y_train.to(self.device)
                output_dict = model.training_step((x_train, y_train), batch_idx)
                loss = output_dict['loss']
                # zero, backpropagation, ajusta parâmetros pelo gradiente descendente
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                outputs.append(output_dict)

            model.training_epoch_end(outputs=outputs)
            
            # Laço de Validação, um a cada época.
            if val_dataloader:
                output_val_end = self.validate(model, val_dataloader)
                print(f'Epoch {i} - {output_val_end["progress_bar"]}')
                # Salvando o melhor modelo de acordo com a loss de validação.
                if output_val_end['valid_loss'] < best_valid_loss:
                    torch.save(model.model.state_dict(), 'best_model.pt')
                    best_valid_loss = output_val_end['valid_loss']

    def validate(self, model, val_dataloader):
        outputs = []
        model.model.eval()
        with torch.no_grad():
            for batch_idx, (x, y) in enumerate(val_dataloader):
                x = x.to(self.device)
                y = y.to(self.device)
                output_dict = model.validation_step((x, y), batch_idx)
                outputs.append(output_dict)

        output_dict = model.validation_epoch_end(outputs=outputs)
        return output_dict

    def test(self, model, test_dataloader):
        outputs = []
        model.model.eval()
        with torch.no_grad():
            for batch_idx, (x, y) in enumerate(test_dataloader):
                x = x.to(self.device)
                y = y.to(self.device)
                output_dict = model.test_step((x, y), batch_idx)
                outputs.append(output_dict)

        output_dict = model.test_epoch_end(outputs=outputs)
        return output_dict

In [ ]:
if pl_original:
    !pip install pytorch_lightning
    from pytorch_lightning import LightningModule, Trainer

## Definindo o Modelo

In [ ]:
models.mobilenet_v2(pretrained=True).eval()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


MobileNetV2(
  (features): Sequential(
    (0): ConvBNActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): ConvBNActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momen

In [ ]:
class Modelo(torch.nn.Module):
    def __init__(self):
        super(Modelo, self).__init__()
        
        num_target_classes = 10
        self.backbone = models.mobilenet_v2(pretrained=True)
        self.backbone.classifier[1].out_features = num_target_classes 

    def forward(self, x):
        return self.backbone(x)

In [ ]:
print(Modelo())

Modelo(
  (backbone): MobileNetV2(
    (features): Sequential(
      (0): ConvBNActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): ConvBNActivation(
            (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1,

## Criação do modelo Pytorch Lightning

In [ ]:
class LightningClassifier(LightningModule):
    def __init__(self, hparams):
        super().__init__()

        self.hparams = hparams
        self.criterion = torch.nn.CrossEntropyLoss(reduction='none')

        # Note como a arquitetura esta dependente dos hiperparâmetros salvos.
        self.model = Modelo()

    def forward(self, x):
        logits = self.model(x)
        preds = logits.argmax(dim=1)
        return logits, preds

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
     
        # predict da rede
        logits = self.model(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        loss = batch_losses.mean()
        run['train/batch_loss'].log(loss)

        # O PL sempre espera um retorno nomeado 'loss' da training_step.
        return {'loss': loss, 'batch_losses': batch_losses}

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()

        run['train/loss'].log(avg_loss)
        return
  
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)

        metrics = {'valid_loss': avg_loss.item(), 'accuracy': accuracy.item()}
        output =  {'progress_bar': metrics, 'valid_loss': avg_loss.item()}
        return output
  
    def test_step(self, val_batch, batch_idx):
        # A implementação deste método é opcional no Pytorch Lightning.
        x, y = val_batch
        
        # predict da rede
        logits, preds = self.forward(x)

        # calcula a perda
        batch_losses = self.criterion(logits, y)
        # calcula a acurácia
        batch_accuracy = (preds == y)
        
        # Retornamos as losses do batch para podermos fazer a média no validation_epoch_end.
        return {'batch_losses': batch_losses, 'batch_accuracy': batch_accuracy}

    def test_epoch_end(self, outputs):
        # A implementação deste método é opcional no Pytorch Lightning.
        avg_loss = torch.stack([output['batch_losses'] for output in outputs]).mean()
        accuracy = torch.stack([output['batch_accuracy'] for output in outputs]).float().mean()

        run['valid/loss'].log(avg_loss)
        run['valid/acuracy'].log(accuracy)
        metrics = {'Test loss': avg_loss.item(), 'test accuracy': accuracy.item()}
        output =  {'progress_bar': metrics}
        return output

    def configure_optimizers(self):
        # Gradiente descendente
        optimizer = torch.optim.SGD(self.model.parameters(), lr=self.hparams['learning_rate'])
        # Aqui usamos um scheduler dummy pois o pytorch lightning original requer um.
        scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 1.0)
        return [optimizer], [scheduler]  # Forma de retorno para associar um otimizador a um scheduler.

## Inicialização dos parâmetros

In [ ]:
hparams = {
    'max_epochs': 10,
    'learning_rate': 0.01,
    'gpus': 1,
}

## Treinamento

In [ ]:
run = neptune.init(project='davi/aula-6', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJlMGU4YzQ5NC1kMzA1LTQ2ZWQtODQ1Ny01ZmQ4ZDc0NTg0M2QifQ==')

pl_model = LightningClassifier(hparams=hparams)
trainer = Trainer(gpus=hparams['gpus'], max_epochs=hparams['max_epochs'])
trainer.fit(pl_model, train_dataloader, val_dataloader)

https://app.neptune.ai/davi/aula-6/e/AUL2-69
Using cuda:0
Epoch 0 - {'valid_loss': 0.3575826585292816, 'accuracy': 0.8774999976158142}
Epoch 1 - {'valid_loss': 0.26524293422698975, 'accuracy': 0.9109999537467957}
Epoch 2 - {'valid_loss': 0.21070700883865356, 'accuracy': 0.9294999837875366}
Epoch 3 - {'valid_loss': 0.2374698966741562, 'accuracy': 0.9223999977111816}
Epoch 4 - {'valid_loss': 0.21904505789279938, 'accuracy': 0.9329999685287476}
Epoch 5 - {'valid_loss': 0.22182108461856842, 'accuracy': 0.9325999617576599}
Epoch 6 - {'valid_loss': 0.22750458121299744, 'accuracy': 0.9361000061035156}
Epoch 7 - {'valid_loss': 0.24441786110401154, 'accuracy': 0.9317999482154846}
Epoch 8 - {'valid_loss': 0.23962032794952393, 'accuracy': 0.9354999661445618}
Epoch 9 - {'valid_loss': 0.23479437828063965, 'accuracy': 0.9372999668121338}


## Teste

In [ ]:
trainer.test(pl_model, test_dataloader)

{'progress_bar': {'Test loss': 0.25956758856773376,
  'test accuracy': 0.9343999624252319}}